# Introduction
## Cartographie avec folium de geopy
The Nominatim API is a REST API (provided by OpenStreetMap), that returns geographic coordinates (latitude and longitude) from a postal address. If several postal addresses (due to inaccuracy or non-existent street number) corresponding to the request are found, the API returns several coordinates, each time with an `importance` score. The coordinates are given in descending order of importance, so you can select only the first address.

In [1]:
# Execute the code below :

import requests
link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
r = requests.get(link).json()
r


[{'place_id': 45768607,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 1203329854,
  'lat': '37.7242361',
  'lon': '15.1789984',
  'class': 'highway',
  'type': 'residential',
  'place_rank': 26,
  'importance': 0.10000999999999993,
  'addresstype': 'road',
  'name': 'Via Pietro Mascagni',
  'display_name': 'Via Pietro Mascagni, Altarello, Giarre, Catania, Sicilia, 95014, Italia',
  'boundingbox': ['37.7234970', '37.7249937', '15.1784912', '15.1794786']},
 {'place_id': 45825211,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 58232256,
  'lat': '37.6091358',
  'lon': '15.1596388',
  'class': 'highway',
  'type': 'residential',
  'place_rank': 26,
  'importance': 0.10000999999999993,
  'addresstype': 'road',
  'name': 'Via Pietro Mascagni',
  'display_name': 'Via Pietro Mascagni, Sciarelle, Acireale, Catania, Sicilia, 95024, Italia',
  'boundingb

In [2]:
# Here we select only the first address (index 0)
print("First address :", r[0])
print("First address longitude :",r[0]['lon'])
print("First address latitude :",r[0]['lat'])


First address : {'place_id': 45768607, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 1203329854, 'lat': '37.7242361', 'lon': '15.1789984', 'class': 'highway', 'type': 'residential', 'place_rank': 26, 'importance': 0.10000999999999993, 'addresstype': 'road', 'name': 'Via Pietro Mascagni', 'display_name': 'Via Pietro Mascagni, Altarello, Giarre, Catania, Sicilia, 95014, Italia', 'boundingbox': ['37.7234970', '37.7249937', '15.1784912', '15.1794786']}
First address longitude : 15.1789984
First address latitude : 37.7242361


In [3]:
# For ease of use, you can add a limit on the number of items returned:

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
r = requests.get(link).json()
print("WITHOUT limit, how many coordinates does this address return?",len(r))

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json&limit=1"
r = requests.get(link).json()
print("WITH limit, how many coordinates does this address return?",len(r))


WITHOUT limit, how many coordinates does this address return? 10
WITH limit, how many coordinates does this address return? 1


## How to create your API query
It's up to you to modify the string to create the right request URL

In [8]:
# We observe that the query consists of a fixed part, followed by the address to be searched for.
# An URL cannot contain a " " space character,
# and special characters or accents should be avoided if possible
import urllib.parse
link_main = 'https://nominatim.openstreetmap.org/?q='
adresse = '54 Via Pietro Mascagni, Catania, Italy'
link_end = '&format=json&limit=1'

link = link_main +  urllib.parse.quote_plus(adresse)

print(link + link_end)


https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni%2C+Catania%2C+Italy&format=json&limit=1


In [ ]:
import requests
import urllib.parse

def get_coordinates(adresse):
    link_main = 'https://api-adresse.data.gouv.fr/search/?q='

    # Remplacer les espaces par des '+'
    adresse = urllib.parse.quote_plus(adresse)

    # Construire l'URL
    url = f"{link_main}{adresse}"

    # Effectuer la requête
    response = requests.get(url)

    # Vérifier que la requête a réussi
    if response.status_code == 200:
        data = response.json()

        # Extraire les coordonnées
        if 'features' in data:
            coordinates = data['features'][0]['geometry']['coordinates'][::-1] # on inverse latitude et longitude
            return coordinates
        else:
            return None
    else:
        print(f"Erreur lors de la requête : {response.status_code}")
        return None


In [42]:
# Create a function here that turns a postal address into a request URL for the Nominatim API,
# then makes the request and returns the coordinates :

import requests
import urllib.parse

def api_address_geopy(adresse):
    link_main = 'https://nominatim.openstreetmap.org/?q='
    link_end = '&format=json&limit=1'

    # Remplacer les espaces par des '+'
    adresse = urllib.parse.quote_plus(adresse)

    # Construire l'URL
    url = f"{link_main}{adresse}{link_end}"
    print(f"\nurl :\n{url} \n")
    # Effectuer la requête
    response = requests.get(url)

    # Vérifier que la requête a réussi
    if response.status_code == 200:
        data = response.json()

        # Extraire les coordonnées
        if 'features' in data:

            coordinates = data['features'][0]['geometry']['coordinates'][::-1] # on inverse latitude et longitude
            print(f"\ncoordinates :\n{coordinates} \n")
            return coordinates
        else:
            return None
    else:
        print(f"Erreur lors de la requête : {response.status_code}")
        return None



In [11]:
# Test it here:
coordinate=API_address(adresse)
coordinate


[42.575417, 9.002372]

# DataViz
Latitude & Longitude can be used on visualization tools, whether they are BI tools (PowerBI, Table), or Python DataViz libraries such as Plotly or Folium.

Here we will display a map with Folium.


In [12]:
# The syntax of Folium is very simple, you start by creating a map centered on a point.
# You can change the default zoom level with the argument "zoom_start".

import folium

# We define a location as a list with 2 values : latitude and longitude.
point = [float(r[0]['lat']), float(r[0]['lon'])]

# We center the map on the location
m = folium.Map(location=point,zoom_start=7)

# We display the map
m


In [13]:
# Then you can add landmarks and put a clickable comment
m = folium.Map(location=point, )
folium.Marker(
    location=point,
    popup='a good restaurant'
    ).add_to(m)
m


# Challenge
Here is a DataFrame with restaurants in Catania, Sicily, and their respective addresses. Here is your mission:
- Create a new column "coordinates", which will store the coordinates corresponding to each address (you can use the function you created previously).
- Display a map with the 4 restaurant markers. Be careful, the restaurants are very close, remember to set the default zoom level so that it is clearly legible. You can center the map on the first restaurant. And display the name of the restaurant in the tooltip popup.

In [52]:
import pandas as pd
restaurants = pd.DataFrame([["Gelateria Zio Pietro dal 1964", "Via Porta di Ferro, 47, 95131 Catania CT"],
                            ["A Casa d'Amici","Via Fischetti, 14, 95131 Catania CT"],
                            ["La Bitta", "Via Acquicella Porto, 95121 Catania CT"],
                            ["Steak House", "Via Porta di Ferro, 8, 95100 Catania CT"]
                            ],
                           columns = ["name", "address"])

restaurants


,name,address
0,Gelateria Zio Pietro dal 1964,"Via Porta di Ferro, 47, 95131 Catania CT"
1,A Casa d'Amici,"Via Fischetti, 14, 95131 Catania CT"
2,La Bitta,"Via Acquicella Porto, 95121 Catania CT"
3,Steak House,"Via Porta di Ferro, 8, 95100 Catania CT"


In [23]:
# !pip install geopy


  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)


In [44]:
# import pandas as pd
# import requests
# from geopy.geocoders import Nominatim
# import folium
# # Initialize Nominatim API
# geolocator = Nominatim(user_agent="geoapiExercises")

# # Function to get coordinates
# def get_coordinates(address):
#     try:
#         location = geolocator.geocode(address)
#         return (location.lat, location.lon)
#     except:
#         return (None, None)



,name,address,coordinates
0,Gelateria Zio Pietro dal 1964,"Via Porta di Ferro, 47, 95131 Catania CT","(None, None)"
1,A Casa d'Amici,"Via Fischetti, 14, 95131 Catania CT","(None, None)"
2,La Bitta,"Via Acquicella Porto, 95121 Catania CT","(None, None)"
3,Steak House,"Via Porta di Ferro, 8, 95100 Catania CT","(None, None)"


In [56]:
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import folium

def api_address_geopy(adresse):
    link_main = 'https://nominatim.openstreetmap.org/?q='
    link_end = '&format=json&limit=1'

    # Remplacer les espaces par des '+'
    adresse = urllib.parse.quote_plus(adresse)

    # Construire l'URL
    url = f"{link_main}{adresse}{link_end}"
    # print(f"\nurl :\n{url} \n")
    # Effectuer la requête
    response = requests.get(url)

    # Vérifier que la requête a réussi
    if response.status_code == 200:
        data = response.json()
        # print(f"\n data:\n{data} \n")
        # Extraire les coordonnées
        if  data:
            coordinates = [float(data[0]['lat']), float(data[0]['lon'])]
            # coordinates = data['features'][0]['geometry']['coordinates'][::-1] # on inverse latitude et longitude
            # print(f"\ncoordinates :\n{coordinates} \n")
            return coordinates
        else:
            return None
    else:
        print(f"Erreur lors de la requête : {response.status_code}")
        return None



In [57]:

# Creation de la colonne  'coordinates'
# restaurants['coordinates'] = restaurants['address'].apply(get_coordinates)
# restaurants['coordinates'] = restaurants['address'].apply(api_address_geopy)
restaurants['coordinates'] = restaurants['address'].apply(api_address_geopy)
restaurants


,name,address,coordinates
0,Gelateria Zio Pietro dal 1964,"Via Porta di Ferro, 47, 95131 Catania CT","[37.5020931, 15.0931951]"
1,A Casa d'Amici,"Via Fischetti, 14, 95131 Catania CT","[37.506171, 15.0943223]"
2,La Bitta,"Via Acquicella Porto, 95121 Catania CT","[37.4894646, 15.0792388]"
3,Steak House,"Via Porta di Ferro, 8, 95100 Catania CT","[37.5029335, 15.0930406]"


In [58]:
#  test des 3  fonctions
# coord\ncoordininate=get_coordinates(restaurants['address'][0])
# print(f"\ncoordinate :\n{coordinate} \n")

# coordinate=API_address(restaurants['address'][0])
# print(f"ate :\n{coordinate} \n")

coordinate=api_address_geopy(restaurants['address'][0])
print(f"\ncoordinate :\n{coordinate} \n")



coordinate :
[37.5020931, 15.0931951] 



In [79]:
import folium

# Assuming 'restaurants' DataFrame already has a 'coordinates' column with the correct data

# Create a Folium map with a default view that will be adjusted later
m = folium.Map( width='80%', height='50%')

# Add markers for each restaurant and collect the coordinates in a list to adjust the map view bounds later
coordinates_list = []
for idx, row in restaurants.iterrows():
    folium.Marker(
        location=row['coordinates'],
        popup=row['name'],
        tooltip=row['name']
    ).add_to(m)
    coordinates_list.append(row['coordinates'])

# Use fit_bounds to automatically adjust the zoom and the centering of the map to include all points
# We add a little padding using the 'padding' parameter to ensure all markers are visible
if coordinates_list:
    m.fit_bounds(coordinates_list, padding=(50, 50))  # You can adjust the padding as needed

# Show the map
m


In [77]:
from geopy.geocoders import Nominatim
import folium

# # Generate map
# m = folium.Map(location=restaurants['coordinates'][0], zoom_start=12)

# Calculate the average coordinates to center the map
average_coords = [restaurants['coordinates'].str[0].mean(), restaurants['coordinates'].str[1].mean()]
# Create a Folium map centered around the average coordinates
m = folium.Map(location=average_coords, zoom_start=14, width='50%', height='50%')

# # Calculate the center of all markers
# latitudes = [coord[0] for coord in restaurants['coordinates']]
# longitudes = [coord[1] for coord in restaurants['coordinates']]
# center = [sum(latitudes) / len(latitudes), sum(longitudes) / len(longitudes)]
# # Generate map
# m = folium.Map(location=center, zoom_start=12)


# Add markers to the map
for idx, row in restaurants.iterrows():
    folium.Marker(
        location=row['coordinates'],
        popup=row['name'],
        tooltip=row['name']
    ).add_to(m)

# Show the map
m


# Remarks on the Nominatim API
As indicated in the quest, there are many resources for geocoding. Most of them are available by registration, and some require a fee.

The Nominatim API is free and without registration. The disadvantage is that it is relatively slow. If you need to use it in the future, remember to store the results so you don't have to run it several times.

For your knowledge, there is also :
- the **reverse** address API, which allows you to find the nearest postal address using geographic coordinates.
- the API from a **CSV file** if you have a lot of addresses to geocode
- the API **GeoJSON** which allows to obtain a geoJSON format of locations to make choropleth maps


All the [documentation is available here](https://nominatim.org/release-docs/develop/api/Search/).